In [1]:
import os
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import requests

In [2]:
# Set the path to the ChromeDriver executable
chrome_driver_path = '/Users/noahlee/Downloads/chromedriver-mac-arm64/chromedriver'

# Initialize the WebDriver service
service = Service(executable_path=chrome_driver_path)

# Initialize the WebDriver with the service
driver = webdriver.Chrome(service=service)

In [3]:
# Set the base URL of the AIX Academy website
base_url = "https://www.g-h.store/"

In [4]:
# List to store all the URLs
urls_to_scrape = [base_url]

In [5]:
# Set to store visited URLs
visited_urls = set()

In [6]:
# Function to get all the links on a page
def get_all_links(url):
    driver.get(url)
    time.sleep(2)  # Wait for the page to load
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    links = soup.find_all('a', href=True)
    page_links = set()
    for link in links:
        href = link['href']
        if href.startswith('/'):
            href = base_url + href.lstrip('/')
        if base_url in href and href not in visited_urls:
            if "/products" not in href:
                page_links.add(href)
    return page_links

In [12]:
# Crawl the website and collect URLs
numpages = len(current_url)
while urls_to_scrape and numpages<800:
    current_url = urls_to_scrape.pop(0)
    if current_url not in visited_urls:
        if "/products" in current_url:
            continue 
        numpages+=1
        visited_urls.add(current_url)
        print(current_url)
        new_links = get_all_links(current_url)
        urls_to_scrape.extend(new_links - visited_urls)

In [13]:
len(visited_urls)
import pickle 
with open('visited_urls.pkl', 'wb') as file:
    pickle.dump(visited_urls, file)

In [14]:
# Create a directory to save the scraped content
if not os.path.exists('text'):
    os.makedirs('text')

In [15]:
# Function to save page content to a .txt file
def save_page_content(url):
    driver.get(url)
    time.sleep(2)  # Wait for the page to load
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    text_content = soup.get_text(separator='\n', strip=True)
    text_content=text_content[:-385]
    file_name = url.replace(base_url, '').replace('/', '_') + '.txt'
    file_path = os.path.join('text', file_name)
    with open(file_path, 'w', encoding='utf-8') as file:
        file.write(text_content)

In [16]:
# Scrape content from each URL and save it
for url in visited_urls:
    save_page_content(url)

# Close the WebDriver
driver.quit()